## BUIDLING A DATABASE FOR CRIME REPORTS
The main purpose of this project is about building a DataBase for Crime Reports in Boston, USA. In this project, i'm heading for creating table with approriate datatype for each column. Also, i'm going to create two Groups for users which are Data analysis and Data engineering Team.

In [1]:
import psycopg2
conn = psycopg2.connect(dbname='dq', user='dq')
conn.autocommit = True  #have to set it for create a DB
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.close()


ProgrammingError: database "crime_db" already exists


In [5]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
conn.autocommit = True
cur = conn.cursor()
#create a schema 
cur.execute("CREATE SCHEMA crimes;")

ProgrammingError: schema "crimes" already exists


In [44]:
import csv
with open('boston.csv') as file:
    reader = csv.reader(file)
    data  = [row for row in reader]
    col_headers = data[0]
    first_row = data[1]
print(len(data))


298330


In [25]:
def get_col_set(csv_file, col_index):
    import csv
    value = set()
    with open('boston.csv') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            value.add(row[col_index])
    return value    

for i in range(len(col_headers)):
    values = get_col_set('boston.csv', i)
    print(col_headers[i], len(values))
    

incident_number 298329
offense_code 219
description 239
date 1177
day_of_the_week 7
lat 18177
long 18177


In [18]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [28]:
#finding max length of description column
description = get_col_set('boston.csv', 2)
len_description = []
for row in description:
    len_description.append(len(row))
print('max_len_description:' , max(list_len))

#finding max length of incident_nubmer

incident_number = get_col_set('boston.csv', 0)
len_incident_max = max([len(row) for row in incident_number])
print('max_len_incident:', len_incident_max)


max_len_description: 58
max_len_incident: 6


In [29]:
#finding the max length of offense code
offense_code = get_col_set('boston.csv', 1)
len_code_max = max([len(row)for row in offense_code])
print('max_offense_code:', len_code_max)

max_offense_code: 4


In [32]:
#finding the max length of lat&long
for i in range(5, 7):
    values = get_col_set('boston.csv', i)
    print(max(values))
#max_attitude : 42.39504158
#max longtitude: -71.17867378

42.39504158
-71.17867378


In [33]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


## Base on value of each column values, there are  datatypes following:
1. incident_number : integer (int4)
2. offense_code :  smallint
3. description : varchar(100)
4. date : date type
5. day_of_the_week: week_enum (enumurated datatype)
6. attitude: decimal
7. longtitude: decimal

In [37]:
#create week_enum for the day_of_the_week column
cur.execute("""
    CREATE TYPE week_enum AS ENUM (
        'Saturday', 'Friday', 'Monday',
        'Wednesday', 'Sunday', 'Tuesday', 'Thursday');
""")
     

ProgrammingError: type "week_enum" already exists


In [38]:
# create table with their data type
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number integer PRIMARY KEY,
        offense_code smallint,
        description varchar(100),
        date date,
        day_of_the_week week_enum,
        attitude decimal,
        longtitude decimal
    );
""")      

In [43]:
#loading data into database
with open('boston.csv') as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
    
    

IntegrityError: duplicate key value violates unique constraint "boston_crimes_pkey"
DETAIL:  Key (incident_number)=(1) already exists.
CONTEXT:  COPY boston_crimes, line 2


In [42]:
#checking data of table
cur.execute("SELECT * from crimes.boston_crimes;")
rows = cur.fetchall()
print(len(rows))

298329


In [46]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

In [47]:
#create readonly group
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("REVOKE ALL ON crimes.boston_crimes FROM readonly;")

cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON crimes.boston_crimes TO readonly;")



In [48]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("REVOKE ALL ON crimes.boston_crimes FROM readwrite;")
cur.execute("GRANT CONNECT ON DATABASE crime_db to readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, UPDATE, INSERT, DELETE ON crimes.boston_crimes TO readwrite;")

In [50]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")


ProgrammingError: role "data_analyst" already exists


In [55]:
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

ProgrammingError: role "data_scientist" already exists


In [57]:
conn.close()

In [73]:
conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()

In [75]:
#checking users and groups
cur.execute("""
    SELECT * 
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist')
    ;
""")
for row in cur:
    print(row)

('dq', 'readonly', 'crime_db', 'crimes', 'boston_crimes', 'SELECT', 'NO', 'YES')
('dq', 'readwrite', 'crime_db', 'crimes', 'boston_crimes', 'INSERT', 'NO', 'NO')
('dq', 'readwrite', 'crime_db', 'crimes', 'boston_crimes', 'SELECT', 'NO', 'YES')
('dq', 'readwrite', 'crime_db', 'crimes', 'boston_crimes', 'UPDATE', 'NO', 'NO')
('dq', 'readwrite', 'crime_db', 'crimes', 'boston_crimes', 'DELETE', 'NO', 'NO')


In [76]:
conn.close()

In [79]:
conn = psycopg2.connect(dbname="crime_db", user="data_analyst", password='secret1')
cur = conn.cursor()
cur.execute("SELECT * FROM crimes.boston_crimes LIMIT 20;")
for row in cur:
    print(row, '\n')

(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', Decimal('42.35779134'), Decimal('-71.13937053')) 

(2, 1402, 'VANDALISM', datetime.date(2018, 8, 21), 'Tuesday', Decimal('42.30682138'), Decimal('-71.06030035')) 

(3, 3410, 'TOWED MOTOR VEHICLE', datetime.date(2018, 9, 3), 'Monday', Decimal('42.34658879'), Decimal('-71.07242943')) 

(4, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.33418175'), Decimal('-71.07866441')) 

(5, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.27536542'), Decimal('-71.09036101')) 

(6, 3820, 'M/V ACCIDENT INVOLVING PEDESTRIAN - INJURY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.29019621'), Decimal('-71.07159012')) 

(7, 724, 'AUTO THEFT', datetime.date(2018, 9, 3), 'Monday', Decimal('42.30607218'), Decimal('-71.0827326')) 

(8, 3301, 'VERBAL DISPUTE', datetime.date(2018, 9, 3), 'Monday', Decimal('42.32701648'), Decimal('-71.10555088')) 

(9, 301, 'ROBBERY - STREET', date